In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'Gender': ['male', 'male', 'male', 'male', 'female', 'female', 'female', 'female'],
    'Height': [6, 5.92, 5.58, 5.92, 5, 5.5, 5.45, 5.75],
    'Weight': [180, 190, 170, 165, 100, 150, 130, 150],
    'Foot_Size': [12, 11, 12, 10, 6, 8, 7, 9]
})
print("Dataset")
print(data)

person = pd.DataFrame({'Height': [5], 'Weight': [130], 'Foot_Size': [6]})
print("\n Instance Dataset")
print(person)

means = data.groupby('Gender').mean()
vars = data.groupby('Gender').var()
priors = data['Gender'].value_counts(normalize=True)
print("\nData mean")
print(means)
print("\nData variance")
print(vars)

def p_x_given_y(x, mean_y, var_y):
    return (1 / np.sqrt(2 * np.pi * var_y)) * np.exp(-(x - mean_y) ** 2 / (2 * var_y))

def calc_prob(person, gender):
    mean = means.loc[gender]
    var = vars.loc[gender]
    prob = priors[gender]
    for feature in person.columns:
        prob *= p_x_given_y(person[feature][0], mean[feature], var[feature])
    return prob

prob_male = calc_prob(person, 'male')
prob_female = calc_prob(person, 'female')

print("Probability Male:", prob_male)
print("Probability Female:", prob_female)
print("Target Label:", "Male" if prob_male > prob_female else "Female")

Dataset
   Gender  Height  Weight  Foot_Size
0    male    6.00     180         12
1    male    5.92     190         11
2    male    5.58     170         12
3    male    5.92     165         10
4  female    5.00     100          6
5  female    5.50     150          8
6  female    5.45     130          7
7  female    5.75     150          9

 Instance Dataset
   Height  Weight  Foot_Size
0       5     130          6

Data mean
        Height  Weight  Foot_Size
Gender                           
female   5.425  132.50       7.50
male     5.855  176.25      11.25

Data variance
          Height      Weight  Foot_Size
Gender                                 
female  0.097500  558.333333   1.666667
male    0.035033  122.916667   0.916667
Probability Male: 2.3136442953578095e-17
Probability Female: 0.0006682947190317349
Target Label: Female
